In [1]:
!jupyter nbextension enable --py widgetsnbextension
!jupyter nbextension install --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbextension` not found.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive 

In [12]:
from ipywidgets import Button, GridBox, Layout, VBox, Output, HBox
import numpy as np
from IPython.display import display, clear_output

class MazeDrawerHTML:
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.grid = np.zeros((rows, cols), dtype=int)  # Maze grid (0: empty, 1: wall)
        self.start_position = None
        self.end_position = None
        self.buttons = []
        self.output = Output()

        self.init_buttons()
        self.init_control_buttons()
    
    def init_buttons(self):
        """Initialize the grid as a set of toggleable buttons with grid lines."""
        cell_size = "20px"  # Adjust cell size for smaller grid
        self.buttons = [
            Button(
                description="",  # No text on the button
                layout=Layout(width=cell_size, height=cell_size, border="1px solid black"),  # Add grid lines
                style={"button_color": "white"},
            )
            for _ in range(self.rows * self.cols)
        ]

        # Assign toggle functionality
        for i, button in enumerate(self.buttons):
            button.on_click(self.make_toggle_handler(i))
        
        # Create a grid layout
        self.gridbox = GridBox(
            children=self.buttons,
            layout=Layout(
                width=f"{self.cols * 25}px",  # Adjust total grid width
                height=f"{self.rows * 25}px",
                grid_template_columns=f"repeat({self.cols}, {cell_size})",
                grid_template_rows=f"repeat({self.rows}, {cell_size})",  # Ensures uniform row spacing
                grid_gap="0px",  # No space between rows or columns
            ),
        )
    
    def make_toggle_handler(self, index):
        """Create a handler to toggle cell state."""
        def handler(button):
            row, col = divmod(index, self.cols)
            
            # If setting start or end position, mark accordingly
            if self.setting_start:
                if self.start_position:
                    self.buttons[self.start_position[0] * self.cols + self.start_position[1]].style.button_color = "white"
                self.start_position = (row, col)
                button.style.button_color = "green"
                self.setting_start = False
            elif self.setting_end:
                if self.end_position:
                    self.buttons[self.end_position[0] * self.cols + self.end_position[1]].style.button_color = "white"
                self.end_position = (row, col)
                button.style.button_color = "red"
                self.setting_end = False
            else:
                # Toggle cell state for maze walls
                self.grid[row, col] = 1 - self.grid[row, col]
                button.style.button_color = "black" if self.grid[row, col] == 1 else "white"
        return handler
    
    def init_control_buttons(self):
        """Initialize control buttons for setting start/end positions and saving."""
        self.setting_start = False
        self.setting_end = False

        self.start_button = Button(
            description="Set Start",
            layout=Layout(width="100px"),
            style={"button_color": "lightblue"},
        )
        self.start_button.on_click(self.set_start)

        self.end_button = Button(
            description="Set End",
            layout=Layout(width="100px"),
            style={"button_color": "lightcoral"},
        )
        self.end_button.on_click(self.set_end)

        self.save_button = Button(
            description="Save Maze",
            layout=Layout(width="100px"),
            style={"button_color": "lightgreen"},
        )
        self.save_button.on_click(self.save_maze)

        self.control_buttons = HBox([self.start_button, self.end_button, self.save_button])
    
    def set_start(self, button):
        """Enable start position setting mode."""
        self.setting_start = True
        self.setting_end = False
        with self.output:
            print("Click a grid cell to set the start position.")

    def set_end(self, button):
        """Enable end position setting mode."""
        self.setting_start = False
        self.setting_end = True
        with self.output:
            print("Click a grid cell to set the end position.")

    def save_maze(self, button):
        """Save the maze to a .txt file and clear the cell's output."""
        filename = "maze_with_positions.txt"
        
        # Save the grid and positions
        with open(filename, "w") as f:
            np.savetxt(f, self.grid, fmt="%d", delimiter="")
            if self.start_position:
                f.write(f"Start: {self.start_position}\n")
            if self.end_position:
                f.write(f"End: {self.end_position}\n")
        
        with self.output:
            clear_output(wait=True)
            print(f"Maze saved to '{filename}' with start and end positions.")
    
    def display(self):
        """Display the interactive maze grid, control buttons, and output."""
        with self.output:
            display(VBox([self.gridbox, self.control_buttons]))
        display(self.output)
    
    def get_maze(self):
        """Get the current maze grid."""
        return self.grid

# Usage example
maze = MazeDrawerHTML(20, 20)
maze.display()

# Access the maze grid state with: maze.get_maze()

Output()